## Cats vs dogs with Resnet
This notebook creates the resnet model to classify the images into cats and dogs.

## Import statements

In [13]:
import os

from resnet50 import Resnet50
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau

## Image width and height
Set image width and height

In [21]:
img_width, img_height = 224, 224

## Filepaths
Set filepaths for training set & validation set

In [3]:
train_path = 'D:/Dogs vs Cats Redux/data/train'
validation_path =  'D:/Dogs vs Cats Redux/data/validation'

Filepath for model weights

In [26]:
weights_path = 'D:/Git Repo/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
save_path = 'D:/Git Repo/Dogs vs Cats Redux/resnet50_weights.h5'

## Create Model
Create model using the Resnet50 class located in Resnet50.py

Refer to:
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

In [5]:
# Create skeleton model and load weights
resnet =  Resnet50(weights_path, size=(img_width, img_height, 3)) # 3 for RGB channel
resnet.make()
resnet.load_weights()

As default Resnet model is trained on Imagenet Class with 10 output classes, we need to fit the last layer with our own custom layer. 

For our problem, we only need to differentiate between two classes (cats vs dogs).Hence it will be a softmax activation layer with only 2 class.

In [6]:
# pop out last layer
resnet.pop()

# freeze previous layers from training
resnet.set_trainable_false()

# add custom softmax layer
resnet.add_layer(2, activation='softmax')

## Hyperparameters for model
Setting the hyperparamters for the Resnet model 
1. learning rate
2. batch size
3. epochs
4. dropout

In [22]:
learning_rate = 0.001
batch_size = 8 # very low since computer GPU cannot support higher size
epochs = 1
dropout = 0 # no dropout

## Creating ImageDataGenerator
Creating the generators to fit into model

In [9]:
train_datagen = image.ImageDataGenerator()

# add image augmentation for training samples
aug_datagen = image.ImageDataGenerator(horizontal_flip=True, 
                             zoom_range=0.1,
                             fill_mode="constant",
                             channel_shift_range=10,
                             rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1)

In [19]:
train_flow = train_datagen.flow_from_directory(train_path,
                                               target_size=(img_width, img_height),
                                               batch_size=batch_size,
                                               shuffle=True,
                                               class_mode='categorical')
val_flow = train_datagen.flow_from_directory(validation_path,
                                             target_size=(img_width, img_height),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             class_mode='categorical')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Optimizers and callbacks

In [14]:
# Adam optimizer
optimizer = Adam(lr=learning_rate)
# Tensorboard visualisation
graph_dir = 'graph/graph_classify'
if not os.path.exists(graph_dir):
    os.makedirs(graph_dir)

# tensorboard
tb = TensorBoard(log_dir=graph_dir, write_graph=True, write_images=True)
# model checkpoint
mc_coord = ModelCheckpoint(filepath='models/classifier_ep{epoch:02d}_loss{val_loss:.2f}_acc{val_accuracy:.2f}.h5',
                           verbose=1,save_best_only=True)
# prevent loss plateau
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0,
                            mode='auto', epsilon=0.0001, cooldown=0, min_lr=1e-7)

## Compile and Fit model

In [23]:
resnet.compile_model(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
resnet.fit_generator(train_flow , val_flow, epochs, [tb, reduceLR])

Epoch 1/1
23000/23000 [==============================] - 331s - loss: 0.0557 - acc: 0.9790 - val_loss: 0.0819 - val_acc: 0.9735

In [27]:
# save weights
resnet.save_weights(save_path)

## Data augmentation
Augment training images to prevent overfitting

In [28]:
train_flow = aug_datagen.flow_from_directory(train_path,
                                               target_size=(img_width, img_height),
                                               batch_size=batch_size,
                                               shuffle=True,
                                               class_mode='categorical')

Found 23000 images belonging to 2 classes.


In [34]:
# set new epochs and learning rate
epochs = 1
learning_rate = 0.00001

In [36]:
resnet.compile_model(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
resnet.fit_generator(train_flow , val_flow, epochs, [tb, reduceLR])

Epoch 1/2
23000/23000 [==============================] - 326s - loss: 0.1152 - acc: 0.9574 - val_loss: 0.0811 - val_acc: 0.9745

In [ ]:
# save weights
resnet.save_weights(save_path)